# 프로토콜(Protocol)
## 객체의 설계도
## 프로토콜의 정의

## 프로토콜 메소드

## 프로토콜에서의 mutating, static 사용

### 스위프트에서는 구조체 내의 메소드가 프로퍼티를 변경하는 경우
- #### 메소드 앞에 반드시 mutating 키워드를 붙여 이 메소드가 프로퍼티 값을 수정하는 메소드임을 표시하도록 강제하고 있다
- #### 이때 그 메소드가 만약 프로토콜에서 선언된 메소드라면 mutating 키워드를 붙이기 위해서는 반드시 프로토콜에 mutating 키워드가 추가되어 있어야 한다

### 클래스와 같은 참조 타입은 mutating 키워드를 붙이지 않아도 메소드 내에서 마음대로 프로퍼티를 수정할 수 있다

### 구조체나 열거형은 프로토콜의 메소드에 mutating 키워드가 추가되어 있지 않을 경우 프로퍼티의 값을 변경할 수 없다
- #### 프로토콜에 선언되지 않은 mutating 키워드를 임의로 구현할 수 없기 때문이다
- #### 만약 억지로 mutating 키워드를 붙여서 메소드를 구현하면 컴파일러는 이를 프로토콜을 구현한 것으로 인정하지 않는다
    - #### 그리하여 구현 대상이 누락되었다는 오류가 발생한다

### 이런 면에서, 프로토콜은 자신을 구현하는 구조체가 마음대로 프로퍼티를 수정하지 못하도록 통제할 수 있는 권한을 가지고 있다
- #### mutating 키워드를 허용하지 않으면, 이를 구현하는 구조체는 메소드 내에서 프로퍼티 값을 수정할 수 없기 때문이다

### 일반적으로 프로토콜에서 메소드 선언에 mutating 키워드가 붙지 않는 것은 다음 두 가지 중 하나로 해석할 수 있다
- #### 1. 구조체나 열거형 등 값 타입의 객체에서 내부 프로퍼티의 값을 변경하기를 원치 않을 때
- #### 2. 주로 클래스를 대상으로 간주하고 작성된 프로토콜일 때

### 프로토콜에서 mutating 키워드를 붙일 때에는 메소드를 표시하는 func 키워드 앞에 mutating을 추가하기만 하면 된다
- #### 프로토콜 메소드에 mutating 키워드가 있으면 이를 구현하는 구조체나 열거형에서도 mutating 키워드를 사용할 수 있다

```
protocol MService {
    mutating func execute(cmd: String)
    func showPort(p: Int) -> String
}

struct RubyMService: MService {
    var paramCommnad: String?
    
    mutating func execute(cmd: String) {
        self.paramCommand = cmd
        if cmd == "start" {
            print("실행합니다")
        }
    }
    
    func showPort(p: Int) -> String {
        return "Port : \(p), now command : \(self.paramCommand!)"
    }
}
```

### 이어서 다음 예제를 보자
- #### 마찬가지로 MService 프로토콜을 구현하는 구조체이지만, execute(cmd:) 메소드에 mutating 키워드가 붙어있지 않다

```
struct RubyMService2: MService {
    var paramCommand: String?
    
    
    func execute(cmd: String) {
        if cmd == "start" {
            print("실행합니다")
        }
    }
    
    func showPort(p: Int) -> String {
        return "Port : \(p), now command : \(self.paramCommand!)"
    }
}
```

### 이 구조체는 프로토콜 메소드에 기재된 mutating 키워드를 사용하지 않았지만, MService 프로토콜을 문제없이 구현하고 있다
- #### 구조체에서 mutating 키워드를 붙이지 않는 것은 실제로 구현된 execute(cmd:) 메소드 내에서 프로퍼티를 변경하지 않기 때문이다

### 프로토콜 쪽에서 mutating 키워드가 추가되어 있다 할지라도 실제 구현하는 쪽에서 프로퍼티의 변경이 없다면 굳이 붙이지 않고 생략해도 오류는 발생하지 않는다

### 이처럼 프로토콜에서 mutating 처리되지 않은 메소드를 구조체에서 임의로 mutating 처리하는 것은 프로토콜의 구현 명세를 위반하는 오류이지만, 그 반대의 경우는 허용된다
- #### 프로토콜에서 mutating 키워드로 선언된 메소드라 하더라도 실제 구현하는 객체에서 필요 없다면 덧붙이지 않아도 된다

### 이를 정리하면 다음과 같다. 이 규칙은 열거형에서 프로토콜을 구현할 때도 동일하게 적용된다

- #### 프로토콜과 구조체 사이의 mutating 키워드 필요 관계

```
프로토콜        구조체        결과
mutating      mutating    OK
mutating      -           OK
-             mutating    ERROR
-             -           OK

* 단, 실제로 구조체의 메소드에서 프로퍼티 값 변경이 없을 때에만
```

### 클래스의 경우 조금 다르다
- #### 클래스는 참조 타입의 객체이므로 메소드 내부에서 프로퍼티를 수정하더라도 mutating 키워드를 붙일 필요가 없다
- #### mutating 키워드가 붙어있는 프로토콜 메소드를 구현할 때도 클래스에서는 프로퍼티의 수정 여부와 관계없이 mutating 키워드를 사용하지 않는다

```
class RubyThread: MService {
    var paramCommand: String?
    
    
    func execute(cmd: String) {
        self.paramCommand = cmd
        
        
        if cmd == "start" {
            print("실행합니다")
        }
    }
    
    
    func showPort(p: Int) -> String {
        return "Port : \(p), now command : \(self.paramCommand!)"
    }
}
```

### 타입 메소드나 타입 프로퍼티도 프로토콜에 정의할 수 있다
- #### 프로토콜의 각 선언 앞에 static 키워드를 붙이면 된다
- #### 클래스에서 타입 메소드를 선언할 때 사용할 수 있는 또 다른 키워드인 class는 프로토콜에서 사용할 수 없다
- #### 프로토콜은 구조체나 열거형, 그리고 클래스에 모두 사용할 수 있는 형식으로 정의되어야 하기 때문이다

### 하지만 프로토콜에서 static 키워드로 선언되었더라도 실제로 클래스에서 구현할 때는 필요에 따라 static이나 class 키워드를 선택하여 사용할 수 있다
- #### 물론 구조체나 열거형에서 구현할 때는 선택의 여자 없이 static 키워드를 붙여야 한다

```
protocol SomeTypeProperty {
    static var defaultValue: String { get set }
    static func getDefaultValue() -> String
}


struct TypeStruct: SomeTypeProperty {
    static var defaultValue = "default"
    
    static func getDefaultValue() -> String {
        return defaultValue
    }
}

class ValueObject: SomeTypeProperty {
    static var defaultValue = "default"
    class func getDefaultValue() -> String {
        return defaultValue
    }
}
```

### 프로토콜이 class 가 아닌 static 키워드를 사용하는 것에 특별한 이유는 없다
- #### 단지 class 키워드가 클래스에 국한된 키워드인 반면 static 키워드는 구조체와 클래스, 그리고 열거형등의 객체가 공통으로 사용하는 키워드이기 때문이다
- #### 따라서 클래스에서 프로토콜을 구현할 때 필요에 따라 static 키워드 대신 class 키워드를 사용하는 것은 프로토콜 명세를 올바르게 구현하는 것으로 간주된다